In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import h5py
import anndata
from anndata import AnnData
from pathlib import Path
import scipy.io as sio
import scanpy as sc
from tqdm import tqdm

## load and filter out data

In [2]:
hest_data_root = '/raid/HEST-1K'
hest_csv = pd.read_csv(os.path.join(hest_data_root, 'HEST_v1_0_0.csv'), index_col = 0)
visium_only = hest_csv[hest_csv['st_technology'] == 'Visium']
visium_only_not_brain = visium_only[visium_only['organ']!='Brain']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Embryo']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Lung/Brain']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Kidney/Brain']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['species'] == 'Homo sapiens']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['nb_genes'] > 10000]
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['id'] !='INT17']
visium_ids = list(visium_only_not_brain['id'])
len(visium_ids)

293

In [3]:
genes = []
for id_ in tqdm(visium_ids):
    d = f'{hest_data_root}/st/{id_}.h5ad'
    adata = sc.read_h5ad(d)
    genes.append(list(adata.var_names))
    del adata

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293/293 [02:01<00:00,  2.42it/s]


In [5]:
visium_ids_take = [visium_ids[0]]
intersection_set = set(genes[0])
# Perform intersection with subsequent lists
for i, gene_list in enumerate(genes[1:]):
    if len(intersection_set&set(gene_list)) > 10000:
        
        intersection_set &= set(gene_list)
        visium_ids_take.append(visium_ids[i+1])
    else:
        print(i, visium_only_not_brain.iloc[i+1]['organ'], len(intersection_set))

# Convert the set back to a list
intersection_list = list(intersection_set)
len(intersection_list)

18 Breast 17368
104 Lung 17197
176 Uterus 17107
177 Uterus 17107
178 Uterus 17107
179 Uterus 17107
180 Uterus 17107
181 Uterus 17107
182 Uterus 17107
183 Uterus 17107
184 Uterus 17107
185 Uterus 17107
186 Uterus 17107
187 Uterus 17107
188 Uterus 17107
189 Uterus 17107
190 Uterus 17107


16958

## create data split

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

split_folder = ''

# Filter the data
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['id'].isin(visium_ids_take)]

# Prepare features and target
X = visium_only_not_brain.drop(columns=['organ'])
y = visium_only_not_brain['organ']

# Identify classes with only one sample
single_instance_classes = y.value_counts()[y.value_counts() == 1].index

# Separate data with single-instance classes
single_instance_data = visium_only_not_brain[visium_only_not_brain['organ'].isin(single_instance_classes)]
remaining_data = visium_only_not_brain[~visium_only_not_brain['organ'].isin(single_instance_classes)]

# Prepare remaining features and target
X_remaining = remaining_data.drop(columns=['organ'])
y_remaining = remaining_data['organ']

# Perform 90:10 split on the remaining data
train_set, val_set = train_test_split(remaining_data, test_size=0.1, stratify=y_remaining, random_state=42)

# Add the single-instance classes to the training set
train_set = train_set.append(single_instance_data)

# Save the split
folds = [(list(train_set['id']), list(val_set['id']))]
np.save(f'{split_folder}/90_10_split.npy', folds)


## HGVs

In [10]:
n_top_genes = 50
hvg_bools = []
for id_ in tqdm(visium_ids_take):
    try:
        d = f'{hest_data_root}/st/{0}.h5ad'.format(id_)
        adata = sc.read_h5ad(d)
        mask = adata.var_names.isin(intersection_list)
        adata = adata[:, mask]

        adata = adata.X.T
    #             print(adata.shape)
        adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

        # Preprocess the data
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)

        #save hvgs
        hvg = adata.var['highly_variable']
    #             print(hvg)
        hvg_bools.append(hvg)
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 276/276 [00:00<00:00, 12014.70it/s]


In [ ]:
hvg_union = hvg_bools[0]
for i in range(1, len(hvg_bools)):
    hvg_union = hvg_union | hvg_bools[i]

print("Number of HVGs: ", hvg_union.sum())

In [ ]:
for id_ in tqdm(visium_ids_take):
    if id_ != 'NCBI855' and id_ != 'NCBI854':
        d = f'{hest_data_root}/st/{0}.h5ad'.format(id_)
        adata = sc.read_h5ad(d)
        adata = adata.X.T 
        adata = adata[hvg_union].toarray()
        print(id_, adata.shape)
        np.save(f"{hest_data_root}/hvg_matrix/hvg_matrix_{0}.npy".format(id_), adata)